[View in Colaboratory](https://colab.research.google.com/github/kintarou/django-starter/blob/master/neural_network_iris.ipynb)

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
%precision 3

'%.3f'

In [0]:
from sklearn.neural_network import MLPClassifier

In [0]:
iris = load_iris()

In [5]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [6]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [7]:
X = iris.data[50:150, 0:2]
y = iris.target[50:150]
print("説明変数の行数・列数:", X.shape)

説明変数の行数・列数: (100, 2)


In [8]:
print("応答変数の行数・列数:", y.shape)

応答変数の行数・列数: (100,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)
print("説明変数の行数・列数:", X_train.shape)
print("応答変数の行数・列数:", y_train.shape)

説明変数の行数・列数: (75, 2)
応答変数の行数・列数: (75,)


In [10]:
y_train[0:10]

array([1, 1, 2, 2, 2, 2, 1, 1, 1, 1])

In [12]:
X_train_df = pd.DataFrame(X_train, columns = ["sepal_len","sepal_wid"])
y_train_df = pd.DataFrame({"species":y_train - 1})
iris_train_df = pd.concat([y_train_df,X_train_df], axis=1)
print(iris_train_df.head(3))

   species  sepal_len  sepal_wid
0        0        5.7        2.8
1        0        6.6        3.0
2        1        6.1        3.0
